# Lesson 5: Indexing & Subsets

## Initial Setup

Import libraries and initialize variables to pick up where we left off in the previous lesson.

In [1]:
import pandas
%matplotlib inline

In [2]:
weather = pandas.read_csv('data/weather_YVR.csv')
weather['Date'] = pandas.to_datetime(weather['Date'])
weather['Rain (inches)'] = weather['Rain (mm)'] / 25.4
weather['T_range (C)'] = weather['T_high (C)'] - weather['T_low (C)']

In [3]:
world = pandas.read_csv('data/gapminder_world_data_2018.csv')
world['area'] = world['population'] / world['pop_density']

# Subsets

We often need to work with subsets of our data, such as specific rows/columns, or values matching some criteria, in order to:
- Analyze and/or plot subsets individually
- Investigate messy and/or suspect data
- Explore potential outliers
- Modify specific values within our data, such as correcting spelling mistakes or filling missing data

> Note: In this workshop, we're focusing mostly on extracting and analyzing subsets *without modifying them* (except for a few simple cases we've seen earlier, such as converting the entire `'Date'` column of weather data from text to datetime type).
- Modifying subsets of `pandas` DataFrames can get tricky due to various issues such chained indexing, views vs. copies of data, and some nitty gritty details that can cause different behaviour (error / warning / assignment failed or succeeded) depending on the exact syntax used &mdash; this topic requires a more in-depth discussion than we have time for here.
- To learn more about modifying data subsets: 
  - [Selecting Subsets of Data in Pandas: Parts 1-3](https://medium.com/dunder-data/selecting-subsets-of-data-in-pandas-6fcd0170be9c)
  - [Selecting Subsets of Data in Pandas: Part 4](https://medium.com/dunder-data/selecting-subsets-of-data-in-pandas-part-4-c4216f84d388)
  - [Understanding SettingWithCopy Warning in Pandas](https://www.dataquest.io/blog/settingwithcopywarning/)

# Selection Methods

When working with Python objects such as lists or dictionaries, we can use square brackets `[]` as an **indexing operator** to select item(s) within the object, for example:
```python
first_five = my_list[:5]
vancouver_population = populations_dict['Vancouver']
```

With `pandas` DataFrames and Series, we still use the indexing operator to select subsets, but it can be applied in many different ways
- `pandas` provides very flexible and powerful options for selecting subsets...
- ... but keeping track of them all can get confusing!

There are three main ways of selecting subsets of a DataFrame (or Series):
<ol>
    <li>Using only square brackets `[]`</li>
    <li>Positional indexing with `.iloc[]`</li>
    <li>Label-based indexing with `.loc[]`</li>
</ol>

We've already used the first approach in the previous lessons. Let's review and then discuss the other two options.

> We'll focus on DataFrames here, but the concepts and syntax are similar for Series

# Square Brackets Only

- With this option, we can select a subset of a DataFrame's rows **or** a subset of a DataFrame's columns, but we can't select both rows and columns at once.
- `pandas` will figure out from the context whether you're selecting rows or columns.


We've seen examples of of this approach already:
- Selecting a column or list of columns of a DataFrame, for example:
```python
population_data = world[['country', 'population']]
```

- Selecting rows of a DataFrame based on some criteria, for example:
```python
long_life = world['life_expectancy'] > 80
world_long_life = world[long_life]
```

- Using just the indexing operator `[]` is a handy shortcut for some very common subsets
- However, it is fairly limited &mdash; for example, we can't select both rows AND columns at once
- For more complex subsets, we can use `.iloc[]` or `.loc[]`, as we shall see shortly

# Indexing

Before we dive into `iloc` and `loc`, let's take a closer look at how data is organized in our DataFrame `world`.

In [4]:
world.head()

,country,population,region,sub_region,income_group,life_expectancy,gdp_per_capita,children_per_woman,child_mortality,pop_density,area
0,Afghanistan,36400000,Asia,Southern Asia,Low,58.7,1870,4.33,65.90,55.7,6.535009e+05
1,Albania,2930000,Europe,Southern Europe,Upper middle,78.0,12400,1.71,12.90,107.0,2.738318e+04
2,Algeria,42000000,Africa,Northern Africa,Upper middle,77.9,13700,2.64,23.10,17.6,2.386364e+06
3,Angola,30800000,Africa,Sub-Saharan Africa,Lower middle,65.2,5850,5.55,81.60,24.7,1.246964e+06
4,Antigua and Barbuda,103000,Americas,Latin America and the Caribbean,High,77.6,21000,2.03,7.89,234.0,4.401709e+02


- As we've seen before, the bold integers on the left are the DataFrame's index.
- The index provides **labels** for each row of a DataFrame.
  - In this case, the labels are just equal to the row numbers.
- We can see the index values by accessing the `index` attribute:

In [5]:
world.index

RangeIndex(start=0, stop=178, step=1)

The output above tells us that the index is a range of consecutive integers from 0 to 177
> Similar to start/stop bounds in list slicing, a `RangeIndex` is inclusive of the start and exclusive of the stop bound.

To better illustrate the difference between positional and label-based indexing, let's create a new DataFrame with the world data, setting the `'country'` column as the index.

In [6]:
world_new = world.set_index('country')
world_new.head()

,population,region,sub_region,income_group,life_expectancy,gdp_per_capita,children_per_woman,child_mortality,pop_density,area
country,,,,,,,,,,
Afghanistan,36400000,Asia,Southern Asia,Low,58.7,1870,4.33,65.90,55.7,6.535009e+05
Albania,2930000,Europe,Southern Europe,Upper middle,78.0,12400,1.71,12.90,107.0,2.738318e+04
Algeria,42000000,Africa,Northern Africa,Upper middle,77.9,13700,2.64,23.10,17.6,2.386364e+06
Angola,30800000,Africa,Sub-Saharan Africa,Lower middle,65.2,5850,5.55,81.60,24.7,1.246964e+06
Antigua and Barbuda,103000,Americas,Latin America and the Caribbean,High,77.6,21000,2.03,7.89,234.0,4.401709e+02


- Notice that the country names are now in bold on the left.
- Instead of integer row numbers, the DataFrame's rows are now labelled with country names:

In [7]:
world_new.index

Index(['Afghanistan', 'Albania', 'Algeria', 'Angola', 'Antigua and Barbuda',
       'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan',
       ...
       'United Kingdom', 'United States', 'Uruguay', 'Uzbekistan', 'Vanuatu',
       'Venezuela', 'Vietnam', 'Yemen', 'Zambia', 'Zimbabwe'],
      dtype='object', name='country', length=178)

# Anatomy of a DataFrame

The diagram below illustrates how data is organized in our DataFrame `world_new` (looking at the first 8 rows).

![dataframe1](img/dataframe1.png)

- The column labels are shaded in green
- The index is shaded in blue &mdash; it provides labels for each row
  - The first row of `world_new` is labelled `'Afghanistan'`
  - The second row is labelled `'Albania'`
  - And so on for each of the following rows
- The actual data is the white area outlined in red

The column labels and index are **metadata** &mdash; they give us information about the data.

The underlying data (without the metadata) is a `numpy` array which we can access with the `values` attribute:

In [8]:
world_new.values

array([[36400000, 'Asia', 'Southern Asia', ..., 65.9, 55.7,
        653500.8976660682],
       [2930000, 'Europe', 'Southern Europe', ..., 12.9, 107.0,
        27383.17757009346],
       [42000000, 'Africa', 'Northern Africa', ..., 23.1, 17.6,
        2386363.6363636362],
       ...,
       [28900000, 'Asia', 'Western Asia', ..., 51.9, 54.8,
        527372.2627737226],
       [17600000, 'Africa', 'Sub-Saharan Africa', ..., 59.5, 23.7,
        742616.0337552743],
       [16900000, 'Africa', 'Sub-Saharan Africa', ..., 55.5, 43.7,
        386727.68878718535]], dtype=object)

As we saw previously, we can access the column and row labels with the `columns` and `index` attributes:

In [9]:
world_new.columns

Index(['population', 'region', 'sub_region', 'income_group', 'life_expectancy',
       'gdp_per_capita', 'children_per_woman', 'child_mortality',
       'pop_density', 'area'],
      dtype='object')

In [10]:
world_new.index

Index(['Afghanistan', 'Albania', 'Algeria', 'Angola', 'Antigua and Barbuda',
       'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan',
       ...
       'United Kingdom', 'United States', 'Uruguay', 'Uzbekistan', 'Vanuatu',
       'Venezuela', 'Vietnam', 'Yemen', 'Zambia', 'Zimbabwe'],
      dtype='object', name='country', length=178)

# Row and Column Positions

DataFrames are 2-dimensional:
- The rows are the first dimension
- The columns are the second dimension

We've seen this with the `shape` attribute, which is a tuple with the number of rows first, followed by the number of columns.

In [11]:
world_new.shape

(178, 10)

Just as the positions of items in a Python list are counted with integers starting at 0, so are the rows and columns of a DataFrame:

![dataframe2](img/dataframe2.png)

> Note: The DataFrame's column labels (green shaded area) are not counted as a row, and the index (blue shaded area) is not counted as a column.

# Positions and Labels

Each individual cell in a DataFrame, such as the yellow one highlighted below, is associated with:
- A row position and column position, and
- A row label and column label

![dataframe3](img/dataframe3.png)

To select data in a DataFrame with `iloc` or `loc`, we specify the **row(s) first, followed by the column(s)**

- With **positional indexing**, we use the `iloc` attribute (you can think of it as "integer location") followed by square brackets containing the row position(s) followed by column position(s)
- For example, we can use the row number and column number to select the yellow cell from the above diagram:

In [12]:
# Position: row 2, column 4
world_new.iloc[2, 4]

77.9

- With **label-based indexing**, we use the `loc` attribute followed by square brackets containing the row label(s) followed by the column label(s)
- For example, we use the row label and column label to select the yellow cell:

In [13]:
# Labels - row: 'Algeria', column: 'life_expectancy'
world_new.loc['Algeria', 'life_expectancy']

77.9

- Label-based indexing can be really handy because it allows us to use our DataFrame as a **lookup table**, as in the example above
- If we don't know or care about the row and column number of the data of interest, we can use labels to look up the value
  - To find out the GDP per capita in Canada, we can just specify the labels `'Canada'` and `'gdp_per_capita'`.

# Exercise 5.1

a) Use `iloc` to select the cell in row 5, column 8 of `world_new`.

b) Same as a) but using `loc`.

c) Use `loc` to select the cell in `world_new` containing the population of Australia.

d) Same as c) but using `iloc`

# Positional Indexing

With Python lists, we can use positional indexing to select:
- A single item at a specified position, or
- A slice `start:stop:step`

In [14]:
animals = ['cat', 'dog', 'elephant', 'duck', 'hamster', 'sheep', 'parrot']

# Print item 0
print(animals[0])

# Print items in the slice 2:5 (i.e. items 2, 3, 4)
print(animals[2:5])

# Print every second item starting from position 0
print(animals[::2])

cat
['elephant', 'duck', 'hamster']
['cat', 'elephant', 'hamster', 'parrot']


We can do the same for rows and columns of a DataFrame with `iloc`.

- Single row and single column, as we saw earlier:

In [15]:
world_new.iloc[1, 8]

107.0

- Slice of rows and a slice of columns:

In [16]:
world_new.iloc[80:85, 4:]

,life_expectancy,gdp_per_capita,children_per_woman,child_mortality,pop_density,area
country,,,,,,
Jordan,76.7,8450,3.24,16.70,112.00,8.839286e+04
Kazakhstan,72.0,24200,2.57,11.50,6.82,2.697947e+06
Kenya,67.3,3110,3.74,47.10,89.50,5.698324e+05
Kiribati,62.2,1890,3.57,52.50,146.00,8.082192e+02
Kuwait,80.0,68400,1.95,7.99,236.00,1.779661e+04


- As with list slicing, slices with `iloc` are **inclusive** of the start bound but **exclusive** of the stop bound
  - Row slice `80:85` is from row 80 up to, but not including row 85
- Also as with list slicing, the start or stop bound can be omitted
  - Column slice `4:` goes from column 4 to the end
  - A slice `:3` would go from position 0 to position 2
- The slice `:` (no start or stop bound) will select all rows or columns

- Several rows and all columns:

In [17]:
world_new.iloc[175:, :]

,population,region,sub_region,income_group,life_expectancy,gdp_per_capita,children_per_woman,child_mortality,pop_density,area
country,,,,,,,,,,
Yemen,28900000,Asia,Western Asia,Low,67.1,2430,3.79,51.9,54.8,527372.262774
Zambia,17600000,Africa,Sub-Saharan Africa,Lower middle,59.5,3870,4.87,59.5,23.7,742616.033755
Zimbabwe,16900000,Africa,Sub-Saharan Africa,Low,60.2,1950,3.61,55.5,43.7,386727.688787


- Slice of several rows and one column:

In [18]:
world_new.iloc[100:108, 1]

country
Malta           Europe
Mauritania      Africa
Mauritius       Africa
Mexico        Americas
Moldova         Europe
Mongolia          Asia
Montenegro      Europe
Morocco         Africa
Name: region, dtype: object

---
### Bonus: Shortcut for Row Selection

When selecting entire rows, we can omit the `:` comma slice
- The following two are equivalent:
  - `world_new.iloc[start:stop:step, :]`
  - `world_new.iloc[start:stop:step]`

In [19]:
world_new.iloc[4:8]

,population,region,sub_region,income_group,life_expectancy,gdp_per_capita,children_per_woman,child_mortality,pop_density,area
country,,,,,,,,,,
Antigua and Barbuda,103000,Americas,Latin America and the Caribbean,High,77.6,21000,2.03,7.89,234.00,4.401709e+02
Argentina,44700000,Americas,Latin America and the Caribbean,High,77.0,18900,2.26,10.60,16.30,2.742331e+06
Armenia,2930000,Asia,Western Asia,Upper middle,76.0,8660,1.60,12.90,103.00,2.844660e+04
Australia,24800000,Oceania,Australia and New Zealand,High,82.9,45800,1.83,3.40,3.22,7.701863e+06


However, we can't omit the `:` row slice when selecting columns with `iloc`. This is because selection with `iloc` always specifies rows first, followed (optionally) by columns.

---

Something we can do with `iloc` that we can't do with Python lists is select multiple items in any arbitrary order.
- We do this by using a list inside the indexing operator
- For example, select rows 4 and 2 (in that order), and slice 3:7 of columns:

In [20]:
world_new.iloc[[4, 2], 3:7]

,income_group,life_expectancy,gdp_per_capita,children_per_woman
country,,,,
Antigua and Barbuda,High,77.6,21000,2.03
Algeria,Upper middle,77.9,13700,2.64


- Select rows 4 and 2, and columns 6, 4, 5:

In [21]:
world_new.iloc[[4, 2], [6, 4, 5]]

,children_per_woman,life_expectancy,gdp_per_capita
country,,,
Antigua and Barbuda,2.03,77.6,21000
Algeria,2.64,77.9,13700


# Exercise 5.2

a) Use `iloc` to select the rows 150 up to and including 160, and the first 3 columns of `world_new`.

b) Use `iloc` to select every tenth row (starting from 0) and columns 2, 0, and 3 of world_new.

# Label-Based Indexing

Similar to `iloc`, label-based indexing with `loc` allows us to select:
- A single cell
- A single row and/or column
- Slices of rows and/or columns
- Multiple rows and/or columns in arbitrary order

We can also select rows or columns with a Boolean Series, and mix and match this with the above selection methods.

However, `loc` has a few important caveats and differences from `iloc`, as we shall see shortly.

- Single row and column:

In [22]:
world_new.loc['Australia', 'population']

24800000

- Slice of rows and slice of columns:

In [23]:
world_new.loc['Canada':'Denmark', 'income_group':'gdp_per_capita']

,income_group,life_expectancy,gdp_per_capita
country,,,
Canada,High,82.2,43800
Central African Republic,Low,51.6,689
Chad,Low,60.5,1860
Chile,High,80.7,23400
China,Upper middle,76.9,16000
Colombia,Upper middle,78.6,13700
Comoros,Low,68.0,1440
"Congo, Dem. Rep.",Low,62.4,751
"Congo, Rep.",Lower middle,63.9,5620


Can you spot the difference compared to slicing with `iloc`?

- Label-based slicing with `loc` is **inclusive** of **both** the start bound **and** the stop bound!
- There are [practical reasons](https://pandas.pydata.org/pandas-docs/stable/advanced.html#endpoints-are-inclusive) why this design decision was made, but we won't get into the details for now.
- It's important to keep this difference in mind when slicing with labels.

- Slice of rows from Spain to Switzerland, with the population density and population columns, in that order:

In [24]:
world_new.loc['Spain':'Switzerland', ['pop_density', 'population']]

,pop_density,population
country,,
Spain,93.00,46400000
Sri Lanka,334.00,21000000
Sudan,23.50,41500000
Suriname,3.64,568000
Swaziland,80.90,1390000
Sweden,24.30,9980000
Switzerland,216.00,8540000


- Select rows with a Boolean Series and columns with a list:

In [25]:
densely_populated = world_new['pop_density'] > 500
world_new.loc[densely_populated, ['region', 'pop_density', 'population']]

,region,pop_density,population
country,,,
Bahrain,Asia,2060.0,1570000
Bangladesh,Asia,1280.0,166000000
Barbados,Americas,666.0,286000
Lebanon,Asia,596.0,6090000
Maldives,Asia,1480.0,444000
Malta,Europe,1350.0,432000
Mauritius,Africa,625.0,1270000
Netherlands,Europe,507.0,17100000
Palestine,Asia,839.0,5050000


# Exercise 5.3

a) Use `loc` to select the GDP per capita and population density in Canada, United States, and Mexico.

b) Use `loc` to select rows where the GDP per capita is greater than `50000`, and a column slice from `'life_expectancy'` through `'children_per_woman'` (inclusive).

[previous lesson](4-sorting-filtering-aggregation.ipynb) | [next lesson](6-visualization-geographic-maps)

[home](0-introduction.ipynb)